In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
import pickle
%matplotlib inline


In [ ]:
#aisles = pd.read_csv("aisles.csv")
#departments = pd.read_csv("departments.csv")
orders = pd.read_csv("orders.csv")
order_prior = pd.read_csv("order_products__prior.csv")
order_train = pd.read_csv("order_products__train.csv")
#products = pd.read_csv("products.csv")

In [ ]:
users_train = orders.user_id[orders.eval_set=='train']
data_train = orders[orders['user_id'].isin(users_train)]
data_train.head()

users_predict = orders.user_id[orders.eval_set=='test']
data_predict = orders[orders['user_id'].isin(users_predict)]
data_predict.head()

In [ ]:
data_train.groupby('user_id').order_number.max().median()

In [ ]:
users_10 = data_train.user_id[data_train.order_number > 9]
data_train = data_train[data_train['user_id'].isin(users_10)]
data_train.info()

In [ ]:
orders_train = data_train.loc[(data_train.user_id==2) & (data_train.eval_set=='prior'),:]
orders_train = pd.merge(orders_train,order_prior,'inner',on='order_id')
orders_test = data_train.loc[(data_train.user_id==2) & (data_train.eval_set=='train'),:]
orders_test = pd.merge(orders_test,order_train,'inner',on='order_id')
print(orders_train.head())
print(orders_test.head())

In [ ]:
order_days = orders_train[['order_number','days_since_prior_order']]
order_days = order_days.drop_duplicates()
order_days.update(order_days[['days_since_prior_order']].fillna(0))
order_days['cum_days'] = order_days['days_since_prior_order'].cumsum() # not an accurate way of thinking about it.
order_days

In [ ]:
order_map = order_days.groupby('order_number').cum_days.mean().to_dict()
orders_train['cum_days'] = orders_train.order_number.map(order_map)
orders_train.head()

In [ ]:
recency_map = orders_train.groupby('product_id').cum_days.max().to_dict()
orders_train['recency'] = (orders_train.cum_days.max() - orders_train.product_id.map(recency_map)) / 7
orders_train.head()

In [ ]:
frequency_map = orders_train['product_id'].value_counts().to_dict()
orders_train['frequency'] = orders_train.product_id.map(frequency_map) / orders_train.order_number.max()
orders_train.head()

In [ ]:
cart_order_map = orders_train.groupby('product_id').add_to_cart_order.mean().to_dict()
orders_train['cart_order'] = orders_train.product_id.map(cart_order_map) / \
                             orders_train.groupby('order_number').order_number.value_counts().mean()
orders_train.head()

In [ ]:
orders_train = orders_train[['product_id','recency','frequency','cart_order']].copy()
orders_train = orders_train.drop_duplicates()
orders_train.head()

In [ ]:
orders_test['buy'] = 1
orders_test = orders_test[['product_id','buy']]
df = pd.merge(orders_train,orders_test,'left',on='product_id')
df.update(df[['buy']].fillna(0))
df = df.drop(['product_id'], axis = 1)
df.head()

In [ ]:
df.buy.value_counts()

In [ ]:
def prepare_rfm(user_ids):
    frames = []
    for user_id in user_ids:
        orders_train = data_train.loc[(data_train.user_id==user_id) & (data_train.eval_set=='prior'),:]
        orders_train = pd.merge(orders_train,order_prior,'inner',on='order_id')
        orders_test = data_train.loc[(data_train.user_id==user_id) & (data_train.eval_set=='train'),:]
        orders_test = pd.merge(orders_test,order_train,'inner',on='order_id')
        frequency_map = orders_train['product_id'].value_counts().to_dict()
        order_days = orders_train[['order_number','days_since_prior_order']]
        order_days = order_days.drop_duplicates()
        order_days.update(order_days[['days_since_prior_order']].fillna(0))
        order_days['cum_days'] = order_days['days_since_prior_order'].cumsum()
        order_map = order_days.groupby('order_number').cum_days.mean().to_dict()
        orders_train['cum_days'] = orders_train.order_number.map(order_map)
        recency_map = orders_train.groupby('product_id').cum_days.max().to_dict()
        orders_train['recency'] = (orders_train.cum_days.max() - orders_train.product_id.map(recency_map)) / 7
        orders_train['frequency'] = orders_train.product_id.map(frequency_map) / orders_train.order_number.max()
        cart_order_map = orders_train.groupby('product_id').add_to_cart_order.mean().to_dict()
        orders_train['cart_order'] = orders_train.product_id.map(cart_order_map) / \
                                     orders_train.groupby('order_number').order_number.value_counts().mean()
        orders_train = orders_train[['product_id','recency','frequency','cart_order']].copy()
        orders_train = orders_train.drop_duplicates()
        orders_test['buy'] = 1
        orders_test = orders_test[['product_id','buy']]
        orders_comb = pd.merge(orders_train,orders_test,'left',on='product_id')
        orders_comb.update(orders_comb[['buy']].fillna(0))
        orders_comb = orders_comb.drop(['product_id'], axis = 1)
        frames.append(orders_comb)
    df = pd.concat(frames)    
    return df

In [ ]:
users_train = data_train['user_id'].unique()
user_ids = list(np.random.choice(users_train, 200))
df = prepare_rfm(user_ids)
df.buy.value_counts()

In [ ]:
features = ['recency','frequency','cart_order']
X = df[features].copy()
y = df['buy'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)
logreg = LogisticRegression(class_weight={0:0.12})
logreg.fit(X_train, y_train)
predictions = logreg.predict(X_test)
y_proba = logreg.predict_proba(X_test)
print('accuracy socre: ', accuracy_score(y_true = y_test, y_pred = predictions))
print('auc: ', roc_auc_score(y_test,predictions))

In [ ]:
orders_test = data_predict.loc[data_predict.eval_set=='test',:]
orders_test = orders_test.iloc[:200,]
features = ['recency','frequency','cart_order']
final = []
for index, row in orders_test.iterrows():
    orders_predict = data_predict.loc[(data_predict.user_id==row.user_id) & (data_predict.eval_set=='prior'),:]
    orders_predict = pd.merge(orders_predict,order_prior,'inner',on='order_id')
    frequency_map = orders_predict['product_id'].value_counts().to_dict()
    order_days = orders_predict[['order_number','days_since_prior_order']]
    order_days = order_days.drop_duplicates()
    order_days.update(order_days[['days_since_prior_order']].fillna(0))
    order_days['cum_days'] = order_days['days_since_prior_order'].cumsum()
    order_map = order_days.groupby('order_number').cum_days.mean().to_dict()
    orders_predict['cum_days'] = orders_predict.order_number.map(order_map)
    recency_map = orders_predict.groupby('product_id').cum_days.max().to_dict()
    orders_predict['recency'] = (orders_predict.cum_days.max() - orders_predict.product_id.map(recency_map)) / 7
    orders_predict['frequency'] = orders_predict.product_id.map(frequency_map) / orders_predict.order_number.max()
    cart_order_map = orders_predict.groupby('product_id').add_to_cart_order.mean().to_dict()
    orders_predict['cart_order'] = orders_predict.product_id.map(cart_order_map) / \
                               orders_predict.groupby('order_number').order_number.value_counts().mean()
    orders_predict = orders_predict[['product_id','recency','frequency','cart_order']].copy()
    orders_predict = orders_predict.drop_duplicates()
    df_predict = orders_predict.copy()
    X = df_predict[features].copy()
    y_proba = logreg.predict_proba(X)
    df_pred = pd.DataFrame({"product_id":df_predict['product_id'],"probability":y_proba[:,1]})
    df_pred = df_pred[df_pred.probability >= 0.5]
    df_pred = df_pred.sort_values(["probability"], ascending = False)
    ord_text = str(row['order_id'])
    prods_text = " ".join(str(x) for x in df_pred['product_id'].values)
    submit_text = ord_text + ", " + prods_text
    final.append([submit_text])

In [ ]:
final[:5]